# Chain of Density Summary

Adams et al. paper: https://arxiv.org/pdf/2309.04269.pdf

# GPT-3.5 Prompt

Adapted from Adams et al. paper.

Modifications:
- Include example JSON
- Ask for similar length summaries, not exact

In [1]:
cod_gpt35_prompt_template = """Article: {{ARTICLE}} 

You will generate increasingly concise, entity-dense summaries of the above Article.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative Entities (";" delimited) from the Article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of similar length which covers every entity and detail from the previous summary plus the Missing Entities.

A Missing Entity is:
- Relevant: to the main story.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the Article.
- Anywhere: located anywhere in the Article.

Guidelines:
- The first summary should be long (4-5 sentences, —80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count: re-write the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Answer in JSON. The JSON should be a list of dictionaries whose keys are "Missing Entities" and "Denser Summary".

e.g.

[
  {
    "Missing Entities": "topic 1; topic 2; topic 3",
    "Denser Summary": "..."
  },
  {
    "Missing Entities": "topic 4; topic 5; topic 6",
    "Denser Summary": "..."
  },
  ...
]
"""

## Analyze GPT-3.5 result

Copy and paste ChatGPT JSON outputs into list below

In [34]:
json_summaries = [
  {
    "Missing Entities": "personal liability; car insurance",
    "Denser Summary": "Home insurance, also known as property and casualty insurance, safeguards your home and belongings against theft, loss, or damage. It may extend coverage to additional living expenses during home unavailability. Condo insurance, distinct from home insurance, covers the inside of your condo and personal liability. Tenant’s insurance, applicable to renters, protects possessions, covers accidental damage, and offers liability coverage. Home-based businesses require separate business insurance, as home insurance may not cover related claims. Understand coverage types, including personal property and liability coverage. Be aware that home insurance excludes predictable events and certain unexpected events. Premiums vary, affected by factors such as residence type, location, and coverage selected."
  },
  {
    "Missing Entities": "home insurance settlement; deductible; loss payee clause",
    "Denser Summary": "Home insurance settlement options involve determining how your claim will be settled. You are responsible for a deductible, an amount you pay before your insurance company covers the rest. Settlements can be based on actual cash value, factoring in depreciation, or replacement value, covering the cost to replace an item. If you have a mortgage, a loss payee clause makes your lender the beneficiary, receiving the claim payout up to the mortgage balance. Your insurance company may choose to pay you directly for repairs or provide funds to your lender, who then disburses the money after you submit receipts as proof."
  },
  {
    "Missing Entities": "property and casualty insurance; loss of business equipment",
    "Denser Summary": "Home insurance, referred to as property and casualty insurance, shields your home and belongings. Condo insurance is distinct, covering the inside of your condo and personal liability. Tenant’s insurance is crucial for renters, covering possessions, accidental damage, and liability. Home-based businesses need separate business insurance, as home insurance may not fully cover business-related claims. Understand personal property and liability coverage. Home insurance excludes predictable and certain unexpected events. Premiums vary based on factors like residence type and location. Settlement options involve actual cash value or replacement value. If you have a mortgage, a loss payee clause makes your lender the beneficiary."
  },
  {
    "Missing Entities": "insurance company; liability coverage; home insurance for unexpected events",
    "Denser Summary": "Home insurance, or property and casualty insurance, safeguards your home and possessions against theft, loss, or damage, providing additional living expense coverage. Condo insurance specifically covers the inside of your condo and personal liability. Tenant’s insurance is essential for renters, offering coverage for possessions, accidental damage, and liability. Separate business insurance is necessary for home-based businesses, as home insurance may not fully cover business-related claims. Understand personal property and liability coverage. Home insurance excludes certain unexpected events. Premiums vary based on factors like residence type and location. Settlement options involve actual cash value or replacement value."
  },
  {
    "Missing Entities": "disaster insurance; insurance works; loss payee clause",
    "Denser Summary": "Home insurance, also known as property and casualty insurance, protects your home and belongings from theft, loss, or damage, with coverage for additional living expenses. Condo insurance is distinct, covering the inside of your condo and personal liability. Tenant’s insurance is crucial for renters, offering coverage for possessions, accidental damage, and liability. Home-based businesses require separate business insurance, as home insurance may not fully cover business-related claims. Understand personal property and liability coverage. Home insurance excludes certain unexpected events. Premiums vary based on factors like residence type and location. Settlement options involve actual cash value or replacement value. A loss payee clause makes your mortgage lender the beneficiary if you have a mortgage."
  },
  {
    "Missing Entities": "home insurance for unexpected events; business insurance coverage",
    "Denser Summary": "Home insurance, also termed property and casualty insurance, safeguards your home and possessions against theft, loss, or damage, with coverage for additional living expenses. Condo insurance specifically covers the inside of your condo and personal liability. Tenant’s insurance is essential for renters, offering coverage for possessions, accidental damage, and liability. Separate business insurance is necessary for home-based businesses, as home insurance may not fully cover business-related claims. Understand personal property and liability coverage. Home insurance excludes certain unexpected events. Premiums vary based on factors like residence type and location. Settlement options involve actual cash value or replacement value."
  }
]


In [35]:
from IPython.display import HTML
import re

def cod_summary_analysis_html(json_summaries):
    html = []
    topic_lists = []
    for i, json_summary in enumerate(json_summaries):
        summary_text = json_summary["Denser Summary"]
        new_topics = json_summary["Missing Entities"]
        new_topics_split = [topic.strip() for topic in new_topics.split(";") if topic.strip()]
        topic_lists.append(new_topics_split)
        for j, topics in enumerate(topic_lists):
            for topic in topics:
                topic_regex = f"[{topic[0].upper()}{topic[0].lower()}]" + topic[1:] # replace leading upper case with regex match
                topic_regex = topic_regex.replace("(", r"\(")
                topic_regex = topic_regex.replace(")", r"\)")
                topic_regex = topic_regex.replace(" ", r"\s")
                summary_text = re.sub(rf"({topic_regex})", rf"<b>\1 [{j+1}]</b>", summary_text)
        html.append(f"<h3>SUMMARY #{i+1}</h3>")
        html.append(f"<div>New Topics: {new_topics}</div><br>")
        html.append(f"<div>{summary_text}</div>")
        html.append("<hr>")
    return html

In [36]:
HTML("\n".join(cod_summary_analysis_html(json_summaries)))

# GPT-4 Prompt

Replicated from Adams et al. paper exactly.

In [5]:
cod_gpt4_prompt_template = """Article: {{ARTICLE}}

You will generate increasingly concise, entity-dense summaries of the above Article.

Repeat the following 2 steps 5 times.

Step 1. Identify 1-3 informative Entities (";" delimited) from the Article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

A Missing Entity is:
- Relevant: to the main story.
- Specific: descriptive yet concise (5 words or fewer).
- Novel: not in the previous summary.
- Faithful: present in the Article.
- Anywhere: located anywhere in the Article.

Guidelines:
- The first summary should be long (4-5 sentences, —80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count: re-write the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.

Remember, use the exact same number of words for each summary.

Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing Entities" and "Denser Summary".
"""

## Fetch and analyze GPT-4 result

In [6]:
from openai import OpenAI
import tiktoken
import json
import os

In [7]:
# os.environ["OPENAI_API_KEY"] = "..."
assert os.getenv("OPENAI_API_KEY") is not None

In [8]:
client = OpenAI()
for model in client.models.list().data:
    if "gpt-4" in model.id:
        print(model)

Model(id='gpt-4', created=1687882411, object='model', owned_by='openai')
Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system')
Model(id='gpt-4-0314', created=1687882410, object='model', owned_by='openai')
Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai')
Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system')


In [37]:
article = """
Home insurance
From: Financial Consumer Agency of Canada

On this page
Understanding home insurance
Condominium insurance
Tenant’s insurance
Insurance for a home-based business
Home insurance coverage
Home insurance premiums
Home insurance settlement options
Settling a claim if you have a mortgage
Understanding home insurance
Home insurance helps protect your home and your belongings in case of theft, loss or damage. It may also help cover additional living expenses if you're temporarily unable to live in your home. These may include living in a hotel or renting a home.

Your home insurance may cover:

damage or loss to your home
damage, theft or loss of your personal possessions
personal property stolen from your vehicle
damage or injury to others who visit your home or property
accidental damage you cause to somebody else’s property
Insurance companies may refer to home insurance as property and casualty insurance. Property and casualty insurance also includes car insurance, business insurance and disaster insurance.

Who your home insurance policy covers
Generally, your home insurance policy covers you and your family members.

If you share your home or if you rent a part of it, you must inform your insurance company.

Check with your insurance company, agent or broker to review who your policy covers.

Condominium insurance
Condominium (condo) insurance is different from home insurance. It covers damage or loss to the inside of your condo and your storage locker. It also covers personal liability claims for someone injured in your condo.

Your condo corporation has an insurance policy that covers the outside structure of the building and its common areas.

Condo insurance may pay for:

damage to the inside structure of your condo
damage to, or loss of, your belongings
damage to improvements you, or the previous owners, made to your condo
damage to other condos or common areas caused by an accident in your condo
additional living expenses, up to a certain amount. For example, if you're temporarily unable to live in your condo because of a loss covered by your policy
Review your condo insurance policy coverage and your condo’s corporation insurance policy coverage.

Tenant’s insurance
You may want to consider buying tenant’s insurance if you live in an apartment or rent a home. Insurance companies may refer to it as renter’s insurance.

You should have enough insurance to cover the cost of replacing everything in your home.

Tenant’s insurance may pay for:

damage to or loss of your possessions if you rent or lease your apartment or home from someone else
personal property stolen from your vehicle
accidental damage you cause to any part of the apartment building or home you're renting
injury to visitors
additional living expenses, within a certain limit. For example, if you're temporarily unable to live in your apartment because of a loss covered by your policy
Insurance for a home-based business
Consider getting business insurance if you have a home-based business.

Home insurance isn't business insurance. If you have a home-based business, you must notify your insurer. Using your home for purposes that your insurer isn't aware of may cancel your policy.

Home insurance generally won't cover claims related to your home-based business. Your home insurance policy may provide limited coverage for damage or loss of business equipment. This coverage is only for the business equipment that is in your home. Business equipment may include computers and power tools. The limited coverage may not be enough to cover the cost of repairing or replacing your business equipment.

Home insurance won't cover:

equipment that is damaged while in your home, if used for business purposes
stolen or damaged equipment while in your vehicle or outside of your home
stolen or damaged tools while in your vehicle or outside of your home
injury to a client while visiting your home, also known as personal liability
Review your home insurance policy to confirm what coverage, if any, it will provide for a home-based business.

Learn more about the types of business insurance coverage.

Home insurance coverage
Make sure you understand what your home insurance covers. You should also understand what types of settlement to expect if you make a claim. Speak with your licensed insurance agent or registered insurance broker.

Types of home insurance coverage
Coverage is the maximum amount of money your insurance company will pay you if you make a claim.

For home insurance, you may need a combination of 2 types of coverage: personal property coverage and liability coverage.

Personal property coverage
Personal property coverage protects you from loss or damage to:

your home or personal possessions
your car
To figure out how much coverage you need, make a list of all your possessions with the following information:

cost to replace the item (replacement value)
make and model of your car
serial number of your car
pictures of your belongings
receipts for major items
This list might be useful if you need to make an insurance claim in the future.

Liability coverage
Liability coverage protects you against legal liability for losses caused by:

injury to other people
damage to the property of others
Read your policy carefully before signing. Make sure you understand what your policy does and doesn’t cover.

Ask your insurance agent or broker about anything you don't understand.

Learn more about the types of home insurance coverage.

What your home insurance policy doesn’t cover
Home insurance doesn’t cover predictable events and certain types of unexpected events.

Unexpected events
Your home insurance may cover certain types of unexpected events or accidents such as fire, windstorm, or theft. However, unexpected events such as earthquakes and floods are usually not covered. You may need to buy additional coverage for these types of events.

Learn more about home insurance for unexpected events and disasters.

Predictable events
Your insurance company doesn’t cover events that they consider predictable and related to the maintenance of your home. For example, suppose you're away for more than 4 days in a row when it's cold outside. Your home insurance may not cover damage to your home because of your pipes freezing.

Home insurance premiums
Premiums are the amount you pay to buy insurance.

When determining how much you'll pay for premiums, insurance companies may consider factors like:

the type of residence you live in, such as a single family home, semi-detached or condo
characteristics of your residence, such as materials it’s made of, age, size, location, replacement value
value of your property and contents they must insure
the distance between your home and a fire hydrant or a fire station
the crime rate in your neighbourhood
your claims history
the type of policy and coverage you selected
the amount of your deductible
A deductible is the amount of your claim that you agree to pay before your insurance company pays the rest.

Shop around for the lowest insurance premiums
Your premiums will vary from one insurance company to another. Shop around, ask for quotes and compare prices before deciding on one insurance company. In some cases, you may be eligible for a discount by combining both your home and car insurance.

In certain provinces and territories, insurance companies may charge higher premiums based on your credit score.

Learn how insurance works.

Home insurance settlement options
Your insurance company will review your claim and decide how it will settle it.

When you make a claim, you're always responsible for paying the deductible. The amount of money you get from your insurance company will depend on your insurance benefits.

Actual cash value
Actual cash value coverage gives you the cost of the item when it was new, minus depreciation. Depreciation is the loss of value due to the age and condition of the item. The depreciation depends on the item insured and the insurance company.

For example, suppose a fire damages your computer. Your insurer will only cover the cost of your computer based on its reduced value at the time of the claim.

Replacement value
Replacement value coverage gives you the actual cost to replace an item following a covered loss.

For example, suppose a fire damaged your computer. You bought your computer 5 years ago for $2,000. Your insurer will cover the cost of buying a new computer of similar quality to the one you lost.

Settling a claim if you have a mortgage
If you have a mortgage, your home insurance policy will usually include a loss payee clause. A loss payee clause makes your mortgage lender your beneficiary. In case of loss or damage to your home, your insurance company will pay your lender (your beneficiary). They’ll pay up to the balance of your mortgage when you submit a claim.

When you make a claim, your insurance company may, at their discretion, give the money to either:

you, so you can fix your home when you submit a claim
your lender, who would then give you the money for repairs or replacement after you submit receipts as proof
"""

In [38]:
prompt = cod_gpt4_prompt_template.replace("{{ARTICLE}}", article)

In [39]:
prompt

'Article: \nHome insurance\nFrom: Financial Consumer Agency of Canada\n\nOn this page\nUnderstanding home insurance\nCondominium insurance\nTenant’s insurance\nInsurance for a home-based business\nHome insurance coverage\nHome insurance premiums\nHome insurance settlement options\nSettling a claim if you have a mortgage\nUnderstanding home insurance\nHome insurance helps protect your home and your belongings in case of theft, loss or damage. It may also help cover additional living expenses if you\'re temporarily unable to live in your home. These may include living in a hotel or renting a home.\n\nYour home insurance may cover:\n\ndamage or loss to your home\ndamage, theft or loss of your personal possessions\npersonal property stolen from your vehicle\ndamage or injury to others who visit your home or property\naccidental damage you cause to somebody else’s property\nInsurance companies may refer to home insurance as property and casualty insurance. Property and casualty insurance al

In [40]:
encoding = tiktoken.encoding_for_model("gpt-4")
num_input_tokens = len(encoding.encode(prompt))

In [41]:
# https://openai.com/pricing#language-models
# GPT-4 costs as of 2023-12

input_cost_per_token = 0.03 / 1_000
output_cost_per_token = 0.06 / 1_000

In [42]:
print(f"Input will cost ${input_cost_per_token * num_input_tokens} ({num_input_tokens} tokens)")

Input will cost $0.06264 (2088 tokens)


In [43]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [44]:
total_cost = input_cost_per_token * completion.usage.prompt_tokens
total_cost += output_cost_per_token * completion.usage.completion_tokens
print(f"Total cost: ${total_cost}")

Total cost: $0.09680999999999998


In [45]:
dict(completion).keys()

dict_keys(['id', 'choices', 'created', 'model', 'object', 'system_fingerprint', 'usage'])

In [46]:
completion.id

'chatcmpl-8QJbp7ZUiQeT8ERgMLayviHh95SV3'

In [47]:
completion.usage

CompletionUsage(completion_tokens=566, prompt_tokens=2095, total_tokens=2661)

In [48]:
print(completion.choices[0].message.content)

[
{
"Missing Entities": "Home insurance; Condominium insurance; Tenant's insurance", 
"Denser Summary": "This piece talks about home insurance, which protects properties and belongings from theft, loss, or damage. Insurance companies may also refer to home insurance as property and casualty insurance. Condominium insurance, another form of property protection, is different from home insurance. It covers internal damage or loss to the condominium or a storage locker. Tenant's insurance, on the other hand, could be considered by those who live in apartments or rent homes."
},
{
"Missing Entities": "Insurance for a home-based business; Liability coverage; Personal property coverage",
"Denser Summary": "Home insurance encompasses property and casualty insurance, including home, condominium, and tenant's insurance. A home-based business, requiring notification to insurers, demands additional business insurance, as home insurance typically excludes business-related claims. Coverage types enc

In [49]:
cod_summary = json.loads(completion.choices[0].message.content)

In [50]:
cod_summary

[{'Missing Entities': "Home insurance; Condominium insurance; Tenant's insurance",
  'Denser Summary': "This piece talks about home insurance, which protects properties and belongings from theft, loss, or damage. Insurance companies may also refer to home insurance as property and casualty insurance. Condominium insurance, another form of property protection, is different from home insurance. It covers internal damage or loss to the condominium or a storage locker. Tenant's insurance, on the other hand, could be considered by those who live in apartments or rent homes."},
 {'Missing Entities': 'Insurance for a home-based business; Liability coverage; Personal property coverage',
  'Denser Summary': "Home insurance encompasses property and casualty insurance, including home, condominium, and tenant's insurance. A home-based business, requiring notification to insurers, demands additional business insurance, as home insurance typically excludes business-related claims. Coverage types enc

In [51]:
print(json.dumps(cod_summary, indent=4))

[
    {
        "Missing Entities": "Home insurance; Condominium insurance; Tenant's insurance",
        "Denser Summary": "This piece talks about home insurance, which protects properties and belongings from theft, loss, or damage. Insurance companies may also refer to home insurance as property and casualty insurance. Condominium insurance, another form of property protection, is different from home insurance. It covers internal damage or loss to the condominium or a storage locker. Tenant's insurance, on the other hand, could be considered by those who live in apartments or rent homes."
    },
    {
        "Missing Entities": "Insurance for a home-based business; Liability coverage; Personal property coverage",
        "Denser Summary": "Home insurance encompasses property and casualty insurance, including home, condominium, and tenant's insurance. A home-based business, requiring notification to insurers, demands additional business insurance, as home insurance typically excludes 

In [52]:
HTML("\n".join(cod_summary_analysis_html(cod_summary)))